In [16]:
# # %load __init__.py
# from FCM import FCM
# from Simulation import simulation


ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "/Users/jshong/opt/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py", line 3326, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-17-0a5ddee752e8>", line 1, in <module>
    gcc --version
NameError: name 'gcc' is not defined

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/Users/jshong/opt/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py", line 2040, in showtraceback
    stb = value._render_traceback_()
AttributeError: 'NameError' object has no attribute '_render_traceback_'

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/Users/jshong/opt/anaconda3/lib/python3.7/site-packages/IPython/core/ultratb.py", line 1101, in get_records
    return _fixed_getinnerframes(etb, number_of_lines_of_context, tb_offset)
  File "/Users/jshong/opt/anacon

NameError: name 'gcc' is not defined

In [3]:
# %load FCM.py
import networkx as nx
import matplotlib.pyplot as plt
from types import FunctionType
import inspect
import sys


'''
This is a Python package for Fuzzy Cognitive Maps
'''

class FCMConstructionError(Exception) :

    def __init__(self,message,errors) :

        message=message+" : "+str(errors)
        super(Exception, self).__init__(message)



class InvalidWeightError(FCMConstructionError) :

    def __init__(self,errors,message="Invalid weight for an edge ") :

        super(InvalidWeightError,self).__init__(message,errors)

class ConceptExistError(FCMConstructionError) :

    def __init__(self,errors,message="Concept does not exist ") :

        super(ConceptExistError,self).__init__(message,errors)

class EdgeExistError(FCMConstructionError) :

    def __init__(self,errors,message="Edge does not exist between ") :

        e=str(errors[0])+" - "+str(errors[1])
        super(EdgeExistError,self).__init__(message,e)



class InvalidConceptValueError(FCMConstructionError) :

    def __init__(self,errors,message="Invalid Concept value ") :

        super(InvalidConceptValueError,self).__init__(message,errors)



class FCM :

    '''
    This is the constructor for the Fuzzy graph.
    It initializes the networkx Digraph
    '''
    def __init__(self) :

        self._fcm_graph=nx.DiGraph()
        self.edgeDict = {}

    '''
    This method is an interface for the add_node
    method of DiGraph
    '''
    def add_concept(self,concept) :

      self._fcm_graph.add_node(concept)
      self._fcm_graph.node[concept]['value']=0.0
      return

    '''
    This method is an interface for the add_edge
    method of Digraph.It checks whether the
    weight provided is the range of [-1,1].
    If the node does not exist,we create them
    before creating the edge.
    '''

    def add_edge(self,concept1,concept2,weight) :

        if weight<-1.0 or weight >1.0 :           # Error checking for the weight

             raise InvalidWeightError(weight)



        if concept1 not in self._fcm_graph.nodes() :   # If the node doesnt exist,create the node
            self.add_concept(concept1)

        if concept2 not in self._fcm_graph.nodes() :   # If the node doesnt exist,create the node
            self.add_concept(concept2)
        #should fix for final version presently would take up memory space that it doesnt need to
        self.edgeDict[(concept1, concept2)] = weight
        self._fcm_graph.add_edge(concept1,concept2,weight=weight) # Adding the edge

    '''
    This method removes edges from the fcm graph.
    It also checks if the nodes exist and if the edge
    exists.
    '''
    def remove_edge(self,node1,node2) :

        if node1 not in self._fcm_graph.nodes()  :
            raise ConceptExistError(node1);

        if  node2 not in self._fcm_graph.nodes() :
            raise ConceptExistError(node2)


        if not self._fcm_graph.has_edge(node1,node2) :
            nodes=[node1,node2]
            raise EdgeExistError(nodes)

        self._fcm_graph.remove_edge(node1,node2)


    '''
    This method is an interface for
    the remove_node() .If the node
    does exist,it prints an error
    message and returns.
    '''
    def remove_concept(self,concept) :

        if concept not in self._fcm_graph.nodes() :

            raise ConceptExistError(concept)



        self._fcm_graph.remove_node(concept)
        return True


    '''
    This method is an interface for
    nodes().It returns the dictionary of
    concepts in the graph having the node of the value as value and the concept as the key.
    '''

    def concepts(self) :
      dictToReturn = {}
      for node in self._fcm_graph.nodes():
       dictToReturn[node] = self._fcm_graph.node[node]['value']
      return dictToReturn

    '''
    This method adds an attribute to
    a node and accepts either an integer
    of a function which returns an integer
    '''

    def set_value(self,concept,num) :

        if concept not in self._fcm_graph.nodes() :   # Error if the given concept does not exist
            raise ConceptExistError(concept)



        if type(num) is int or type(num) is float  :             # If the parameter passed is an int,add it to the attribute

          if num>=-1.0 and num<=1.0 :
            self._fcm_graph.node[concept]['value']=num
          else :
            raise InvalidConceptValueError(num)

        elif type(num) is FunctionType or type(num) is self.FunctionType :

            param_length=(inspect.getargspec(num)[0])


            if len(param_length)!=0 :
                raise InvalidConceptValueError(num)

            self._fcm_graph.node[concept]['value']=num()

        else :
            raise InvalidConceptValueError(num)





    '''
    This method is an interface for the draw()
    in the networkx package.We draw the DiGraph
    using spring layout and labels with the help
    of matplotlib
    '''
    def draw(self) :

       nx.draw(self._fcm_graph,pos=nx.spring_layout(self._fcm_graph),with_labels=True)
       plt.show()
       
       
    '''
    This method returns a dict with the edge combinations and their weights    
    '''
    def getEdges(self):
        return self.edgeDict

In [14]:
# %load Simulation.py
"""
Created on Mon Oct 31 21:21:39 2016

@author: Eric
"""
from copy import deepcopy
import numpy as np
from sys import maxsize
from networkx import to_numpy_matrix
from FCM import *
import time


class InputTypeError(Exception) :

    def __init__(self,message,errors) :
        message=message+" : "+str(errors)
        super(Exception, self).__init__(message)

class InvalidValueError(Exception) :

    def __init__(self,message,errors) :
        message=message+" : "+str(errors)
        super(Exception, self).__init__(message)





class simulation:

    '''
    This class is responsible for the simulation of the Fuzzy Cognitive Maps.
    It has a method called 'run()' which can run Regular Simulation 

    '''


    def __init__(self, FCM,converge_concepts_dict=None):
        '''
        Constructor
        The constructor takes the following parameters
        1) FCM object
        2) a dictionary with key as concept and value as a tuple of size 2
        '''


        ''' an fcm object '''
        self.fcm = deepcopy(FCM)
        self.numSteps = maxsize
        #should look for a way to do this without storing both the keys and the dict. More for convenience in checking later and acces
        self.stabilizers = []
        self.stableDict = {}
        self.converge_concepts_dict=converge_concepts_dict
        self.transferFunction = np.tanh
        self.edgeMatrix = to_numpy_matrix(self.fcm._fcm_graph).transpose() #influence to a node in stored in the row
        self.stable_concepts = self._stable_concepts() #indexes of concepts that should not be changed by the transfer function

    def stabilize(self, concept, threshold):


        '''
        stabilize
        parameters: concept: A valid concept in the fcm
        threshold: A threshold that states a difference of this amount means stable
        Returns: void
        Description: We check that a valid concept in the fcm has been input and add it to the dictionary of all stabilizers.
        if the stabilizer is already in the list of stabilizers we just set the new threshold
        '''




        if concept not in self.fcm.concepts():
            print("Please input a valid concept")
            return
        else:
            if concept not in self.stabilizers:
                self.stabilizers.append(concept)
                self.stableDict[concept] = threshold

            else:
                self.stableDict[concept] = threshold
    def steps(self,numsteps):

        '''
        A function which takes input as a number of steps
        '''


        if type(numsteps) is not int:
            raise InputTypeError("Invalid steps value type")
        if numsteps > 0 and numsteps < maxsize:
            self.numSteps = numsteps

        else:
            print( "Please input a positive number of steps that is less than ", sys.maxsize)
            return
    def changeTransferFunction(self,function):

        '''
        changeTransferFunction
        Parameters: function (function with 1 argument): a function that takes one argument and maintains values in range [-1,1]
        return: void
        Description: Changes the transfer function. error check will pass 100 and -100  to see if they stay in range
        '''

        if callable(function):
            if function(100) > 1 or function(-100) < -1:
                raise InvalidValueError("Error Transfer function must keep values in range [-1,1]")
            else:
                self.transferFunction = function
        else:
            raise InputTypeError("Must pass a function")

    def _updateNodes(self, nodeValues,c = None): #nodevalues is a list of the node values
        '''
        updateNodes(should only be called by run method)
        parameters: nodevalues(iterable list): a list in node order that matches the edge matrix
        c(optional double): a weight to modify the values of the nodes
        returns: the updated values of the nodes after one time step
        Description: will convert the list into a numpy array and multiply it with the edge list to get the changes to each node.
        the changes will be applied. Applies all nodes to the transfer function. If a node has no incoming edges it is declared
        stable and returned to its old value
        '''

        values_vector = np.asarray(nodeValues) #make list into a numpy vector
        update = np.dot(self.edgeMatrix,values_vector)# get new vector of values to be added
        if c is not None:
            if type(c) is not float:
                raise InvalidValueError("Weight c needs to be a decimal value")
            values_vector = values_vector*c
        newValues = np.add(values_vector, update) #values after addition
        newValList = newValues.tolist()[0] #convert to list
        #only apply if hs an incom,ing edge
        newNodeValueTrans = [self.transferFunction(x) for x in newValList] #applies transfer function to each value
        if self.stable_concepts is not None:
            for index in self.stable_concepts:
                newNodeValueTrans[index] = nodeValues[index]
        return newNodeValueTrans

    def _is_Stable(self, oldNodes, newNodes): #oldNods and newNodes are lists
        '''
        is_stable(should only be called by run function)
        Arguments: oldNodes(iterable list): list of node values from before the timestep update
        newNodes(iterable list): list of node values from after the time step
        Returns: Boolean
        Description: Checks the absolute value of the difference between the old and new values of the nodes to see if they are
        less than the designated threshold
        '''

        for node in self.stabilizers:
            index = self.fcm._fcm_graph.nodes().index(node)
            if abs(newNodes[index] - oldNodes[index]) >= self.stableDict[node]:
                return False

        return True

    def run(self,c = None):

        '''
        Run
        Arguments: c (optional): weight for the nodes to be adjusted by in the time step updates
        returns: the dictionary of the new concept values
        Description: Will run the simulation until either time step limit is hit or is stable
        Will print out the number of steps and the nodes with their correlated values
        '''


        count = 0
        returnList = []
        oldValues = []
        nodeOrder = self.fcm._fcm_graph.nodes() #edge matrix is in order of nodes
        for node in nodeOrder:
                oldValues.append(self.fcm.concepts()[node])

        while count < self.numSteps:
            returnList.append(self._makeDict(oldValues))
            newValues = self._updateNodes(oldValues,c)

            if self._is_Stable(oldValues,newValues):
                break

            oldValues = newValues

            count += 1
#        newValues =
       # self._output_results(newValues,count)
        if count == self.numSteps:
            print ("Max number of steps used. Values may not be stable")

       # print 'Total time taken ='+(str(end_time-start_time))+' seconds'

        return returnList


    def _output_results(self, conceptValues, steps):

        '''
        output_result(should only be called by run)
        Parameters: conceptValues(iterable list): list of the final concept values in nodeOrder
        steps(int): the number of steps the simulation runs for
        returns: void
        Description: Will relate the concept values with the concepts due to node order. will print out the
        1)number of steps
        2)the initial concept values
        3) the final concept values
        '''
        
        outDict = self._makeDict(conceptValues)
        print ( "The number of Steps was: ", steps)
        print ("The Initial concept Values were: \n", self.fcm.concepts())
        print ("The final concept Values were: \n", outDict)
        #return outDict

    def _stable_concepts(self):

        '''
        stable_concepts(called on init)
        arguments: none
        Returns: iterabl List: List of the index positions for nodes to remain stable
        Description: Checks which nodes have no incoming edges and marks them as to remain stable throughout the simulation
        '''

        stableList = []
        index = 0
        for node in self.fcm._fcm_graph.nodes():
            if self.fcm._fcm_graph.in_degree(node) == 0:
                stableList.append(index)
            index += 1
        return stableList


    def _makeDict(self, values):
        '''
        _makeDict
        Arguments: values(List): list of concept values in node order
        returns: A dictionary of concepts with their corresponding values
        Description: Turns the lists of concept values in node order into a dictionary
        and returns the dictionary. Only call in methods
        '''
        outDict = {}
        index = 0
        #nodeOrder guaranteed to be the same size as concept values and in the order needed
        for node in self.fcm._fcm_graph.nodes():
            outDict[node] = values[index]
            index += 1

        return outDict
